<a href="https://colab.research.google.com/github/kanishk-24/Save-to-GDrive/blob/main/Save_torrent_to_Google_Drive_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Python3 and libtorrent

In [ ]:
!python -m pip install --upgrade pip setuptools wheel

In [ ]:
!python -m pip install lbry-libtorrent

In [ ]:
!apt install python3-libtorrent
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

# Add Torretn File

Your default download path is set to "My Drive/Downloads/Torrents" folder. You can change that in 'save_path:' below 

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

# Add Magnet

Your default download path is set to "My Drive/Downloads/Torrents" folder. You can change that in 'save_path:' below 

In [ ]:
params = {"save_path": "/content/drive/My Drive/Downloads/Torrents"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

# Start

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)